In [61]:
!pip install pyspark

In [189]:
import pandas as pd
import random
import os
import sys
from pyspark.context import SparkContext
from pyspark.conf import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.types import *
import pyspark.sql.functions as F
import pyspark.sql.types as pst
from pyspark.ml.feature import Bucketizer
import copy
import pandas as pd
from sklearn.metrics import roc_auc_score
from tqdm import tqdm

**促活场景，可以不发优惠券**

In [190]:
user_id = [3790260]*4 + [4687825]*4 + [5077519]*4 + [7927329]*4
coupon_id = [2060114740, 1806204031, 4096652575, 1234832524] * 4
coupon_reduce = [20, 10, 10, 0] * 4
coupon_threshold = [60, 60, 40, -1] * 4
is_convert = []
control_convert = []

In [191]:
for _ in range(4):
  for _ in range(3):
    is_convert.append(random.uniform(0.005, 0.3))
  temp = random.uniform(0, 0.005)
  is_convert.append(temp)
  control_convert.extend([temp]*4)

In [192]:
data = {'user_id': user_id, 'coupon_id': coupon_id, 'coupon_reduce': coupon_reduce, 'coupon_threshold': coupon_threshold, 'is_convert': is_convert, 'control_convert':control_convert}

In [193]:
df_old = pd.DataFrame(data)
df_old['expeted_uplift'] = (df_old['is_convert'] - df_old['control_convert']) * df_old['coupon_threshold']

In [194]:
df_old

,user_id,coupon_id,coupon_reduce,coupon_threshold,is_convert,control_convert,expeted_uplift
0,3790260,2060114740,20,60,0.123163,0.004987,7.090579
1,3790260,1806204031,10,60,0.031058,0.004987,1.564253
2,3790260,4096652575,10,40,0.046526,0.004987,1.661554
3,3790260,1234832524,0,-1,0.004987,0.004987,-0.000000
4,4687825,2060114740,20,60,0.238143,0.000583,14.253599
5,4687825,1806204031,10,60,0.097641,0.000583,5.823499
6,4687825,4096652575,10,40,0.289030,0.000583,11.537874
7,4687825,1234832524,0,-1,0.000583,0.000583,-0.000000
8,5077519,2060114740,20,60,0.125512,0.001945,7.414024
9,5077519,1806204031,10,60,0.040139,0.001945,2.291634


**拉新场景，都需要发优惠券**

In [195]:
user_id = [3790260]*4 + [4687825]*4 + [5077519]*4 + [7927329]*4
coupon_id = [2060114740, 1806204031, 4096652575, 1234832524] * 4
coupon_reduce = [20, 10, 10, 0] * 4
coupon_threshold = [98, 98, 68, -1] * 4
is_convert = []
control_convert = []

In [196]:
for _ in range(4):
  for _ in range(3):
    is_convert.append(random.uniform(0.005, 0.3))
  temp = 0
  is_convert.append(temp)
  control_convert.extend([temp]*4)

In [197]:
data = {'user_id': user_id, 'coupon_id': coupon_id, 'coupon_reduce': coupon_reduce, 'coupon_threshold': coupon_threshold, 'is_convert': is_convert, 'control_convert':control_convert}

In [198]:
df_new = pd.DataFrame(data)
df_new['expeted_uplift'] = (df_new['is_convert'] - df_new['control_convert']) * df_new['coupon_threshold']

In [199]:
df_new

,user_id,coupon_id,coupon_reduce,coupon_threshold,is_convert,control_convert,expeted_uplift
0,3790260,2060114740,20,98,0.234391,0,22.970321
1,3790260,1806204031,10,98,0.291884,0,28.604671
2,3790260,4096652575,10,68,0.095974,0,6.526213
3,3790260,1234832524,0,-1,0.000000,0,-0.000000
4,4687825,2060114740,20,98,0.007300,0,0.715354
5,4687825,1806204031,10,98,0.291711,0,28.587705
6,4687825,4096652575,10,68,0.101929,0,6.931201
7,4687825,1234832524,0,-1,0.000000,0,-0.000000
8,5077519,2060114740,20,98,0.192216,0,18.837196
9,5077519,1806204031,10,98,0.006500,0,0.637004


**转为pyspark计算**

In [200]:
# 初始化spark会话
spark = SparkSession.builder.getOrCreate()

In [201]:
spark_df_old = spark.createDataFrame(df_old)
spark_df_old.show()

+-------+----------+-------------+----------------+--------------------+--------------------+-------------------+
|user_id| coupon_id|coupon_reduce|coupon_threshold|          is_convert|     control_convert|     expeted_uplift|
+-------+----------+-------------+----------------+--------------------+--------------------+-------------------+
|3790260|2060114740|           20|              60| 0.12316319313373365|0.004986870849788...|  7.090579337036706|
|3790260|1806204031|           10|              60|0.031057757985655486|0.004986870849788...| 1.5642532281520158|
|3790260|4096652575|           10|              40| 0.04652571646806576|0.004986870849788...| 1.6615538247310881|
|3790260|1234832524|            0|              -1|0.004986870849788...|0.004986870849788...|               -0.0|
|4687825|2060114740|           20|              60| 0.23814274635997612|5.827660486510183E-4| 14.253598818679507|
|4687825|1806204031|           10|              60|  0.0976410841304231|5.82766048651018

In [202]:
spark_df_new = spark.createDataFrame(df_new)
spark_df_new.show()

+-------+----------+-------------+----------------+--------------------+---------------+------------------+
|user_id| coupon_id|coupon_reduce|coupon_threshold|          is_convert|control_convert|    expeted_uplift|
+-------+----------+-------------+----------------+--------------------+---------------+------------------+
|3790260|2060114740|           20|              98| 0.23439103142214346|              0| 22.97032107937006|
|3790260|1806204031|           10|              98|  0.2918844029483058|              0| 28.60467148893397|
|3790260|4096652575|           10|              68| 0.09597372071105824|              0|  6.52621300835196|
|3790260|1234832524|            0|              -1|                 0.0|              0|              -0.0|
|4687825|2060114740|           20|              98|0.007299530146938252|              0|0.7153539543999486|
|4687825|1806204031|           10|              98|  0.2917112767914632|              0|28.587705125563392|
|4687825|4096652575|        

In [203]:
result_df_new = spark_df_new.groupby('user_id').agg(
    F.collect_list('is_convert').alias('is_convert'),
    F.collect_list('coupon_id').alias('coupon_id'),
    F.collect_list('coupon_reduce').alias('coupon_reduce'),
    F.collect_list('coupon_threshold').alias('coupon_threshold'),
    F.collect_list('expeted_uplift').alias('expeted_uplift'),).cache()

result_df_new.show()

+-------+--------------------+--------------------+---------------+----------------+--------------------+
|user_id|          is_convert|           coupon_id|  coupon_reduce|coupon_threshold|      expeted_uplift|
+-------+--------------------+--------------------+---------------+----------------+--------------------+
|3790260|[0.23439103142214...|[2060114740, 1806...|[20, 10, 10, 0]|[98, 98, 68, -1]|[22.9703210793700...|
|5077519|[0.19221628886193...|[2060114740, 1806...|[20, 10, 10, 0]|[98, 98, 68, -1]|[18.8371963084695...|
|4687825|[0.00729953014693...|[2060114740, 1806...|[20, 10, 10, 0]|[98, 98, 68, -1]|[0.71535395439994...|
|7927329|[0.29986478954906...|[2060114740, 1806...|[20, 10, 10, 0]|[98, 98, 68, -1]|[29.3867493758085...|
+-------+--------------------+--------------------+---------------+----------------+--------------------+



In [204]:
result_df_old = spark_df_old.groupby('user_id').agg(
    F.collect_list('is_convert').alias('is_convert'),
    F.collect_list('coupon_id').alias('coupon_id'),
    F.collect_list('coupon_reduce').alias('coupon_reduce'),
    F.collect_list('coupon_threshold').alias('coupon_threshold'),
    F.collect_list('expeted_uplift').alias('expeted_uplift'),).cache()

result_df_old.show()

+-------+--------------------+--------------------+---------------+----------------+--------------------+
|user_id|          is_convert|           coupon_id|  coupon_reduce|coupon_threshold|      expeted_uplift|
+-------+--------------------+--------------------+---------------+----------------+--------------------+
|3790260|[0.12316319313373...|[2060114740, 1806...|[20, 10, 10, 0]|[60, 60, 40, -1]|[7.09057933703670...|
|5077519|[0.12551203406948...|[2060114740, 1806...|[20, 10, 10, 0]|[60, 60, 40, -1]|[7.41402410811466...|
|4687825|[0.23814274635997...|[2060114740, 1806...|[20, 10, 10, 0]|[60, 60, 40, -1]|[14.2535988186795...|
|7927329|[0.23376322316685...|[2060114740, 1806...|[20, 10, 10, 0]|[60, 60, 40, -1]|[13.7732520659361...|
+-------+--------------------+--------------------+---------------+----------------+--------------------+



**MCKP求解**

In [211]:
epoch = 1000
B = 50
C = 3 * B
alpha = 1 / B
lr_decay = 0.98

In [212]:
def MCKP(df, epoch, B, C, alpha, lr_decay=1):
    N = df.count()
    lambda_B = 0
    lambda_C = 0

    for _ in tqdm(range(epoch)):

        temp = df.cache()

        # 固定lambda，更新xij，选择收益最大的优惠券
        temp = temp.withColumn('score', F.expr('zip_with(is_convert, coupon_reduce, (x, y) -> (x - %f * y))' % (lambda_B)))
        temp = temp.withColumn('score', F.expr('zip_with(score, expeted_uplift, (x, y) -> (x + %f * y))' % (lambda_C)))
        temp = temp.withColumn('choice', F.expr('array_position(score, array_max(score)) - 1'))

        # 固定优惠券，更新lambda
        delta_B = temp.select(F.sum(F.col('coupon_reduce')[F.col('choice')])).collect()[0][0]
        lambda_B = max(0, lambda_B - alpha * (B - delta_B)/N)

        delta_C = temp.select(F.sum(F.col('expeted_uplift')[F.col('choice')])).collect()[0][0]
        lambda_C = max(0, lambda_C - alpha * (delta_C - C)/N)

        if _ % 100 == 0:
            print("epoch: {0}, lambda_B: {1}, lambda_C: {2}".format(_, lambda_B, lambda_C))

        alpha *= lr_decay
        temp.unpersist()

    return temp

In [213]:
result_old = MCKP(result_df_old, epoch, B, C, alpha, lr_decay)

  0%|          | 1/1000 [00:02<39:22,  2.36s/it]

epoch: 0, lambda_B: 0.05, lambda_C: 0.5684925203371937


 10%|█         | 101/1000 [04:25<40:30,  2.70s/it]

epoch: 100, lambda_B: 6.352429626889687, lambda_C: 23.42621565852137


 20%|██        | 201/1000 [09:36<50:06,  3.76s/it]

...

 80%|████████  | 801/1000 [36:51<10:49,  3.26s/it]

epoch: 800, lambda_B: 7.30663058522406, lambda_C: 26.923640062694737


 90%|█████████ | 901/1000 [42:40<04:37,  2.81s/it]

epoch: 900, lambda_B: 7.306631195262946, lambda_C: 26.92364224802779


100%|██████████| 1000/1000 [47:06<00:00,  2.83s/it]


In [214]:
result_old = result_old.withColumn('choose_coupon', F.col('coupon_id')[F.col('choice')])

In [215]:
result_old.show()

+-------+--------------------+--------------------+---------------+----------------+--------------------+--------------------+------+-------------+
|user_id|          is_convert|           coupon_id|  coupon_reduce|coupon_threshold|      expeted_uplift|               score|choice|choose_coupon|
+-------+--------------------+--------------------+---------------+----------------+--------------------+--------------------+------+-------------+
|3790260|[0.12316319313373...|[2060114740, 1806...|[20, 10, 10, 0]|[60, 60, 40, -1]|[7.09057933703670...|[44.8947699266866...|     0|   2060114740|
|5077519|[0.12551203406948...|[2060114740, 1806...|[20, 10, 10, 0]|[60, 60, 40, -1]|[7.41402410811466...|[53.6054303143421...|     0|   2060114740|
|4687825|[0.23814274635997...|[2060114740, 1806...|[20, 10, 10, 0]|[60, 60, 40, -1]|[14.2535988186795...|[237.864328805708...|     0|   2060114740|
|7927329|[0.23376322316685...|[2060114740, 1806...|[20, 10, 10, 0]|[60, 60, 40, -1]|[13.7732520659361...|[224.92

In [216]:
result_new = MCKP(result_df_new, epoch, B, C, alpha, lr_decay)

  0%|          | 1/1000 [00:03<56:18,  3.38s/it]

epoch: 0, lambda_B: 0, lambda_C: 0.2389737228881316


 10%|█         | 101/1000 [04:35<36:42,  2.45s/it]

epoch: 100, lambda_B: 2.125082088057853, lambda_C: 9.713371213149808


 20%|██        | 201/1000 [09:04<35:45,  2.69s/it]

...

 80%|████████  | 801/1000 [37:10<08:19,  2.51s/it]

epoch: 800, lambda_B: 2.4499997655627324, lambda_C: 11.161973714480649


 90%|█████████ | 901/1000 [41:32<04:02,  2.44s/it]

epoch: 900, lambda_B: 2.449999968909026, lambda_C: 11.161974621073217


100%|██████████| 1000/1000 [45:59<00:00,  2.76s/it]


In [217]:
result_new = result_new.withColumn('choose_coupon', F.col('coupon_id')[F.col('choice')])

In [218]:
result_new.show()

+-------+--------------------+--------------------+---------------+----------------+--------------------+--------------------+------+-------------+
|user_id|          is_convert|           coupon_id|  coupon_reduce|coupon_threshold|      expeted_uplift|               score|choice|choose_coupon|
+-------+--------------------+--------------------+---------------+----------------+--------------------+--------------------+------+-------------+
|3790260|[0.23439103142214...|[2060114740, 1806...|[20, 10, 10, 0]|[98, 98, 68, -1]|[22.9703210793700...|[207.628540661323...|     1|   1806204031|
|5077519|[0.19221628886193...|[2060114740, 1806...|[20, 10, 10, 0]|[98, 98, 68, -1]|[18.8371963084695...|[161.452530554090...|     0|   2060114740|
|4687825|[0.00729953014693...|[2060114740, 1806...|[20, 10, 10, 0]|[98, 98, 68, -1]|[0.71535395439994...|[-41.007937514689...|     1|   1806204031|
|7927329|[0.29986478954906...|[2060114740, 1806...|[20, 10, 10, 0]|[98, 98, 68, -1]|[29.3867493758085...|[279.31